In [145]:
import requests
import bs4
from bs4 import BeautifulSoup
import statsmodels as sm
import pandas as pd
import urllib
import simplejson
import urllib2
import re
from tqdm import tqdm

#--------------------------------------------------------------------------------------------------------------    
#Defining function to extract latitude and longitude using googlemaps API:
googleGeocodeUrl = 'http://maps.googleapis.com/maps/api/geocode/json?'

def get_coordinates(query, from_sensor=False):
    query = query.encode('utf-8')
    params = {
        'address': query,
        'sensor': "true" if from_sensor else "false"
    }
    url = googleGeocodeUrl + urllib.urlencode(params)
    json_response = urllib.urlopen(url)
    response = simplejson.loads(json_response.read())
    if response['results']:
        location = response['results'][0]['geometry']['location']
        latitude = location['lat']
        longitude = location['lng']
        lat.append(latitude)
        lon.append(longitude)
        print query, latitude, longitude
    else:
        latitude = None
        longitude = None
        lat.append(0)
        lon.append(0)

#--------------------------------------------------------------------------------------------------------------    
#Extracting the MTR stations list from wikipedia website:        
table = []
station_name = []
wiki = "https://en.wikipedia.org/wiki/List_of_MTR_stations"
header = {'User-Agent': 'Mozilla/5.0'} 
req = urllib2.Request(wiki,headers=header)
page = urllib2.urlopen(req)
soup = BeautifulSoup(page)

table = soup.findAll("table")
for tr in table:
    for i in tr.findAll("a"):
        if i.get("title") != None:
            station_name.append(i.get("title"))

Estaciones = []
for i in station_name:
    if "Station" in i.split():
        Estaciones.append(i)

#--------------------------------------------------------------------------------------------------------------    
#Droping MTR stations that are duplicated and removing future MTR stations: 

df = pd.DataFrame(Estaciones)
df = df.drop_duplicates()
df["station"] = df[0]
df = df.iloc[:, 1:]
lista = list(df.station.unique())
estacio = lista[:-31]
new_estacio = lista[-31:]

#Cleaning the name of the station, removing MTR word:
df1 = pd.DataFrame(estacio)
df1["estacion"] = df1[0]
del df1[0]
df1.estacion = df1.estacion.str.rstrip("(MTR)")

#Adding Hong Kong to the station name, making sure that googlemaps API will check in the correct place:
listaestaciones =  list(df1.estacion)
listaestaci = []
for i in listaestaciones:
    listaestaci.append(str(i) +", Hong Kong")
    
#--------------------------------------------------------------------------------------------------------------    
#Using googlemaps API, extract latitude and longitude of each one of the MTR stations: 
lat = []
lon = []
for i in tqdm(listaestaci):
    get_coordinates(i, from_sensor=False)
    
#Create a dataframe with the name of the station and coordinates, saving it into a csv file "MTR_List":
dfmtr = pd.DataFrame({"name": listaestaci, "lat" : lat,"lon" : lon})
dfmtr = dfmtr[["name", "lat", "lon"]]
dfmtr.to_csv("datasets/MTR_List")